## Librerías:

In [ ]:
# Instalación de librerías necesarios
import sys
import logging, io, json, warnings

logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

!pip install unidecode
!pip install es-lemmatizer
!python -m spacy download es_core_news_md
!python -m spacy link es_core_news_md es --force

python = sys.executable

!{python} -m pip install -U rasa_nlu==0.15.1 rasa_nlu[spacy]==0.15.1 sklearn_crfsuite

✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_md')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/es_core_news_md -->
/usr/local/lib/python3.6/dist-packages/spacy/data/es
You can now load the model via spacy.load('es')
Requirement already up-to-date: ktrain in /usr/local/lib/python3.6/dist-packages (0.21.2)
Requirement already up-to-date: rasa_nlu==0.15.1 in /usr/local/lib/python3.6/dist-packages (0.15.1)
Requirement already up-to-date: sklearn_crfsuite in /usr/local/lib/python3.6/dist-packages (0.3.6)


In [ ]:
import nltk
import numpy as np
import pandas as pd
import re
import spacy
import pickle
import string
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from random import choice
from unidecode import unidecode
from es_lemmatizer import lemmatize

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
stopwords = stopwords.words('spanish')
snowball = SnowballStemmer('spanish')
nlp = spacy.load('es')
nlp.add_pipe(lemmatize, after='tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Datasets - Intents y Utterances

In [ ]:
#@title nlu-config.yml
%%writefile nlu-config.yml
language: "es"

pipeline:
- name: "SpacyNLP"
- name: "SpacyTokenizer"
- name: "SpacyFeaturizer"
- name: "RegexFeaturizer"
- name: "CRFEntityExtractor"
- name: "EntitySynonymMapper"
- name: "SklearnIntentClassifier"

Writing nlu-config.yml


In [ ]:
#@title intents.txt
%%writefile intents.txt
## saludo
- hola
- hola que tal
- buenas tardes
- buenos dias
- buenas noches
- hola, tengo una consulta
- quisiera consultar algo
- hola quiero saber si me pueden ayudar
- que tal tengo una consulta

## despedida
- hasta luego
- hasta pronto
- fue un gusto conversar
- super bueno el apoyo
- por el momento no gracias
- no deseo gracias
- chau, muchas gracias
- fue útil la información
- nos vemos
- chau
- adiós
- gracias por todo
- eso es todo muchas gracias
- gracias por la ayuda
- me sirvió de mucho el apoyo
- gracias por la información
- en otro momento les escribo
- no deseo nada más
- es todo por mi parte

## sintoma_alergia
- tengo los ojos irritados
- tengo los ojos rojos
- tengo los ojos llorosos
- picazon de ojos
- dolor de ojos
- me duelen ambos ojos
- tengo muchos estornudos
- nariz tapada
- nariz con mocos
- tengo la nariz llena de mocos
- dolor de garganta
- me duele mucho la garganta
- picazon de garganta
- carraspera
- tengo tos constante

## sintoma_anemia
- sensación de cansancio
- estoy muy cansado
- mareos
- tengo mareos
- me siento mareado
- aturdimiento
- estoy aturdido
- me he sentido aturdido
- debilidad
- me siento debil
- me siento muy débil
- fatiga
- estoy fatigado
- me siento fatigado
- energía disminuida
- me siento con poca energía
- no tengo energía
- piel pálida
- estoy pálido
- me veo pálido
- palidez
- piel amarillenta
- tengo la piel amarrilla
- mi piel esta amarilla
- el color de mi piel es amarillo
- palpitaciones
- siento palpitaciones extrañas
- el corazón me palpita de forma irregular
- mi corazon palpita de forma rara
- latidos del corazón irregulaes
- siento latidos del corazon irregulaes
- creo que tengo frecuencia cardiaca alta
- tengo frecuencia cardiaca alta
- frecuencia cardiaca alta
- dificultad para respirar
- me cuesta mucho respirar
- me falta el aire constantemente
- dolor en el pecho
- siento dolor en el pecho
- mis manos y pies están fríos
- siento mis pies fríos
- mis manos están frías
- manos y pies fríos
- dolores de cabeza

## sintoma_gripe
- me duele la cabeza
- me duele mucho la cabeza
- tengo dolor de cabeza
- dolor de cabeza
- dolor muscular
- me duelen los musculos
- me duele todo el cuerpo
- tengo dolor muscular 
- escalofríos
- siento escalofríos
- me dan escalofríos
- pérdida de apetito
- no tengo apetito
- dolor de cuerpo
- dolor generalizado de cuerpo
- fiebre
- tengo fiebre
- tengo temperatura 
- tengo temperatura alta
- congestión nasal
- tengo mocos
- tengo la nariz con mocos
- nariz con mocos
- tengo secreción nasal
- secreción nasal
- estornudos
- estoy con estornudos
- tos
- tengo tos
- dolor de gargánta
- me duele la garganta
- me duele mucho la garganta
- sudoración
- sudo mucho
- ganglios linfáticos inflamados
- mis ganglios linfáticos están inflamados
- me duele la garganta
- me duelen mis ganglios
- náuseas
- tengo nauseas
- me dan ganas de vomitar
- ganas de vomitar
- sensasión de vómitos
- presión en el pecho
- siento presión en el pecho
- me presiona el pecho

## sintoma_asma
- dificultad para respirar
- falta de aire
- dolor de pecho
- opresión en el pecho
- tos
- pitido al respirar
- silbido al respirar

## sintoma_alzeimer
- pérdida de orientación
- últimamente he perdido el sentido de la orientación
- me olvido del lugar en el que estoy
- suelo olvidarme dónde estoy
- a veces no se donde estoy
- confusión de lugares
- suelo confundir los lugares
- agitación
- me siento agitado
- me agito rapidamente
- cambios de humor
- tengo cambios de humor repentino
- me cuesta planificar
- me es dificil planificarme
- dificultad para planificar
- a veces me cuesta planificar mis cosas
- dificultad para concentrarse
- no puedo concentrarme
- a veces me cuesta concentrarme
- dificultad para resolver problemas
- a veces tengo dificiltad para resolver problemas
- dificultad para recordar eventos
- me cuesta recordar eventos
- me falla la memoria
- no me acuerdo de algunas cosas
- capacidad menor para cuidarse a sí mismo
- no puedo cuidarme por mi cuenta
- problemas para encontrar palabras al hablar
- a veces me trabo y no se qué decir 
- me cuesta decir lo que pienso al hablar
- depresión repentina
- me siento deprimido
- ultimamente me siento deprimido

## sintoma_embarazo
- Ausencia de periodos menstruales
- no he tenido mi periodo menstrual
- hinchazón y sensibilidad en las mamas
- siento dolor en las mamas
- hinchazon en las mamas
- sensibilidad en las mamas 
- nauseas y vómitos
- tengo nauseas
- tengo ganas de vomitar
- antojos
- tengo antojos repentinos
- aversiones a la comida
- no soporto el olor a comida
- no me gusta el olo de la comida
- fatiga
- siento fatiga
- me siento cansado
- estoy consada
- dolores de cabeza
- me duele la cabeza
- cambios en el estado de ánimo
- micción frecuente
- voy al baño frecuentemente
- ganas de ir al baño constantemente

## sintoma_hipertiroidismo
- Frecuencia cardiaca rápida
- me late el corazón muy rápido
- el corazón me late muy de prisa
- sudoración excesiva
- sudo mucho
- intolerancia al calor
- no soporto el calor
- no soporto altas temperaturas
- me fastidia el calor
- temblores
- siento temblores en el cuerpo
- nerviosismo o agitación
- me agito mucho
- me siento agitado constantemente
- fatiga
- me siento fatigado
- siento fatiga
- pérdida de peso
- he bajado de peso muy rápido
- he bajado de peso
- he perdido peso
- caída del cabello
- se me cae el cabello
- se me cae el pelo
- pierdo cabello
- me estoy quedando sin pelo
- mucho apetito
- tengo mucha hambre
- tengo más hambre de lo normal
- problemas de concentración
- me cuesta concentrarme
- deposiciones frecuentes
- voy al baño constantemente
- voy al baño a cada rato
- voy al baño frecuentemente 

## sintoma_hipotiroidismo
- fatiga
- me siento fatigado
- siento fatiga
- depresión
- me siento deprimido
- estoy con depresión
- leve aumento de peso
- he subido de peso
- intolerancia al frío
- no soporto el frío
- últimamente no soporto el frío
- no puedo estar en lugares fríos
- somnolencia
- tengo muchas ganas de dormir
- me cuesta manenerme despierto en el día
- tengo mucho sueño durante el día
- estreñimiento
- estoy estreñido
- cabello seco y áspero
- mi cabello está seco
- mi cabello está áspero
- mi pelo está seco
- mi pelo está áspero
- piel seca y calambres musculares
- tengo la piel seca
- tengo calambres musculares
- calambres en los músculos
- niveles de colesterol elevados en la sangre
- tengo el colesterol elevado
- tengo el nivel de colesterol alto

## sintoma_diabetes
- mucha sed
- tengo mucha sed
- tomo mucha agua durante el día
- me siendo deshidratado
- deshidratación
- estoy deshidratado
- hambre
- tengo mucha hambre
- no siendo saciedad
- ganas de miccionar
- tengo muchas ganas de ir al baño
- voy al baño muchas veces durante el día
- me dan ganas de ir al baño muchas veces en el día
- fatiga
- me siento fatigado
- siento fatiga
- visión borrosa
- veo borroso
- no veo bien
- pérdida de peso inesperada
- he bajado de peso de forma repentina
- mucha hambre
- tengo mucha hambre
- aumento de apetito
- llagas de curación lenta
- mis heridas se curan lentamente
- infecciones frecuentes
- tengo infecciones frecuentes
- encías rojas e inflamadas
- encías inflamadas
- encías rojas
- hormigueo o entumecimiento en las manos o los pies
- sensación de entumecimiento en los pies
- sensación de entumecimiento en las manos
- sensación de hormigueo en las manos
- sensación de hormigueo en los pies

## sintoma_sida
- fiebre
- inflamación de los ganglios linfáticos
- ganglios inflamados
- dolores musculares y articulares
- dolor de cabeza
- dolor de garganta
- escalofríos
- sudoración nocturna
- úlceras en la boca

## sintoma_hipertension
- mareos
- dificultad para respirar
- dolor de cabeza fuerte
- náuseas
- vómitos
- visión borrosa
- hemorragias nasales
- sangre en la orina
- fatiga
- sangrado nasal

## sintoma_artritis
- dolor muscular
- dolor articular
- inflamación articular
- rigidez articular
- fatiga
- falta de apetito
- pérdida de energía
- articulaciones hinchadas

## sintoma_hepatitisB
- fatiga
- náuseas
- dolor en el abdomen
- dolor abdominal
- orina oscura
- dolor articular
- pérdida de apetito
- debilidad y fatiga
- piel amarilla
- parte blanca del ojo amarilla

## sintoma_dengue
- dolor de cabeza
- fiebre
- agotamiento
- dolor severo en las articulaciones de los músculos
- inflamación de los ganglios linfáticos
- erupción cutánea
- dolor en los ojos

## sintoma_malaria
- fiebre
- escalofríos
- náuseas
- vómitos
- heces con sangre
- sudoración

## sintoma_varicela
- debilidad general
- fiebre muy alta
- manchas rojas en la piel
- manchas rojas y fiebre
- cansancio y sensación de malestar general

## sintoma_ictericia
- ojos amarillos
- decoloración amarilla de la piel
- coloración amarilla de la piel
- piel clara
- coloración amarilla dentro de la boca
- somnolencia excesiva
- orina oscura o de color marrón
- heces pálidas
- heces de color arcilla

## sintoma_diarrea
- deposiciones flojas y acuosas
- calambres abdominales
- dolor de estómago
- calambres y dolor abdominal
- fiebre
- sangre en las heces
- hinchazón
- náuseas

## sintoma_catarata
- disminución de la claridad de la visión
- incapacidad para ver con luz tenue
- pérdida de sensibilidad al contraste
- pérdida de la visión
- ver halos alrededor de las luces
- deslumbramiento perturbador en la luz
- visión borrosa

## sintoma_neumonia
- fiebre
- resfriado
- escalofrios
- tos
- dificultad para respirar
- fatiga

## sintoma_retencionurinaria
- incapacidad para orinar
- flujo de orina es débil e interrmpido
- malestar en la parte inferior del abdomen
- dolor en la parte inferior del abdomen
- hinchazón de la parte inferior del abdomen

## sintoma_insomnia
- cansancio
- fatiga diurna
- cambios de humor
- poca concentración
- ansiedad
- dolor de cabeza
- falta de energía

## sintoma_covid
- fiebre
- tos seca
- cansancio
- molestias y dolores
- dolor de garganta
- diarrea
- conjuntivitis
- pérdida del sentido del olfato o del gusto
- dificultad para respirar
- falta de aire
- dolor o presión en el pecho
- incapacidad para hablar o moverse

Writing intents.txt


In [ ]:
#@title utterances.txt
%%writefile utterances.txt
## saludo
- Hola! Mi nombre es MedCheck. ¿Listo para agendar tu siguiente teleconsulta?
- Un gusto conocerte! Mi nombre es Medcheck, tu asistente de teleconsulta.
- Buen día! Me llamo Medcheck, tu asistente de teleconsulta.
- Hola! Soy tu asistente de teleconsulta, MedCheck.
- Un gusto saludarte! Mi nombre es Medcheck, y estoy aquí para ayudarte agendando tu siguiente teleconsulta.

## solicita_datos_inicio
- Para continuar, por favor completa los siguintes datos.
- Listos para comenzar! por favor completa los siguintes datos.
- Para comenzar, por favor completa los siguintes datos.

## solicita_dni
- Ingresa el número de tu Documento Nacional de Identificación (DNI):

## solicita_nombre
- Ingresa tu nombre:

## solicita_apellidos
- Ingresa tus apellidos:

## solicita_reingresar_datos
- Por favor vuelve a ingresar tus datos, el formato no es correcto.

## ingreso_datos_inicio_ok
- Perfecto! gracias por compartir la información, seguimos!
- Gracias por la información, continuamos!

## solicita_sintoma
- Para seleccionar los mejores especialistas para tu caso, por favor dinos tus síntomas. Procura que tu respuesta sea concisa.
- Para entender mejor lo que necesitas, por favor compartenos tus síntomas. Procura que tu respuesta sea concisa.

## solicita_datos_final
- Para terminar, por favor completa los siguintes datos.
- Ya casi terminamos! por favor completa los siguintes datos.

## solicita_correo
- Ingresa tu dirección de correo electrónico:

## solicita_celular
- Ingresa tu número de celular:

## solicita_datos_final_ok
- Listo! gracias por compartir la información.
- Gracias por la información!

## solicita_confirmación_final
- ¿Estás conforme con reservar la cita?.

## confirmación_final
- Todo en orden! Te llegará un correo con el link para tu teleconsulta.

## despedida
- Espero que todo haya sido de tu agrado, nos vemos!
- Muchas gracias por confiar en nosotros, hasta luego!
- Todo listo para tu cita! Un gusto conversar contigo.
- Un gusto conversar contigo, nos vemos!
- Eso fue rápido! Tu cita está agendada. Hasta luego!


Writing utterances.txt


## Pre procesamiento de dataset (NLP)

In [ ]:
def read_file(file_name):
	"""Leemos el archivo y mapeamos los datos a un diccionario"""
	content = {}
	file_content = open(file_name, "r")
	for line in file_content:
		if line.startswith('##'):
			name = line[2:].strip()
			if name not in content:
				content[name] = []
		elif line.startswith('-'):
			content[name].append(preprocess(line[1:]))
	file_content.close()
	return content

def remove_punctiations(text):
	return re.sub('[%s]' % re.escape(string.punctuation), '',text)
 
def remove_accent(text):
	"""Removemos los acentos de las palabras"""
	return unidecode(text).strip()

def convert_to_lower(text):
  """Cambiamos todos nuevos tokens a minusculas"""
  text = text.lower()
  return text

def preprocess(text):
	"""Realizamos el pre procesamiento de los texts"""
	text = remove_punctiations(text)
	text = remove_accent(text)
	text = convert_to_lower(text)
	return text

def remove_stopwords(text):
	return [word for word in nltk.word_tokenize(text) if (word not in stopwords) and re.search('[a-zA-Z]', word)]

def stem_word(word, stemmer):
	return stemmer.stem(word)
 
def lemmatize_text(text, lemmatizer):
	text = remove_stopwords(text)
	text = ' '.join(text)
	lemmas = []
	for token in lemmatizer(text):
		lemmas.append(remove_accent(token.lemma_))
	return ' '.join(lemmas)

def stem_text(text, stemmer):
	tokens = remove_stopwords(text)
	text = [stem_word(word, stemmer) for word in tokens]
	return ' '.join(text)

def create_dataset(content):
	X, y = [], []
	for key, values in content.items():
		X.extend(values)
		y.extend([key for _ in range(len(values))])
	return X, y

In [ ]:
#Leemos el archivo de Intents.txt
intents = read_file("intents.txt")
intents_x, intents_y = create_dataset(intents)

In [ ]:
# Almacenamos la información en Pandas
dataset = pd.DataFrame([intents_x, intents_y]).T
dataset = dataset.rename(columns={0: 'sintomas', 1: 'intent' })
#dataset['sintomas_stemm'] = [stem_text(row_sintoma, snowball) for row_intent, row_sintoma in zip(dataset['intent'], dataset['sintomas']) if row_intent != 'saludo' or row_intent != 'despedida']
#dataset['sintomas_lemma'] = [lemmatize_text(row_sintoma, nlp) for row_intent, row_sintoma in zip(dataset['intent'], dataset['sintomas']) if row_intent != 'saludo' or row_intent != 'despedida']

dataset['sintomas_stemm'] = [stem_text(row, snowball) for row in dataset['sintomas']]
dataset['sintomas_lemma'] = [lemmatize_text(row, nlp) for row in dataset['sintomas']]
dataset['sintomas_pre'] = [' '.join(remove_stopwords(row)) for row in dataset['sintomas']]

## Generación de Machine Learning (NLU)

In [ ]:
vectorizer = TfidfVectorizer()
model_tree = DecisionTreeClassifier(max_depth=14)
model_forest = RandomForestClassifier(max_depth=14)
model_logistic = LogisticRegression(solver='newton-cg', multi_class='multinomial')

In [ ]:
encoder = LabelEncoder()
x = vectorizer.fit_transform(dataset['sintomas_stemm']) #'sintomas_stemm'
y = pd.get_dummies(dataset['intent'])
y_encoder = encoder.fit_transform(dataset['intent'])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y_encoder, test_size=0.1, stratify=y)

In [ ]:
model_tree.fit(x_train, y_train)
model_forest.fit(x_train, y_train)
model_logistic.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
print(model_tree.score(x_test, y_test) * 100)
print(model_forest.score(x_test, y_test) * 100)
print(model_logistic.score(x_test, y_test) * 100)

17.073170731707318
43.90243902439025
51.21951219512195


In [ ]:
with open('model_vector_tfidf.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

In [ ]:
with open('model_decision_tree.pkl', 'wb') as f:
    pickle.dump(model_tree, f)

In [ ]:
with open('model_decision_tree.pkl', 'rb') as m:
    model = pickle.load(m)
    print(model)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=14, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')


## Generación de Respuesta del Bot (NLG)

In [ ]:
def is_symptom_key(intent, keys):
    intent_type = intent.split('_')[0]
    for k in keys:
        ks = k.split('_')[-1]
        if ks == intent_type:
            return (True, k)
    return (False, None)

def choose_utterance(intent, utterances):
    is_symptom, rintent = is_symptom_key(intent, utterances.keys())
    if intent in utterances.keys():
        answers = utterances[intent]
        return choice(answers)
    elif is_symptom: 
         answer = utterances[rintent]
         return choice(answer)
    else:    
        return 'No entiendo'

## Pruebas de predicción

In [ ]:
utterances = read_file('utterances.txt')

In [ ]:
# preprocesamos el texto
text = "Prodría realizar una consulta"
text = preprocess(text)
text = stem_text(text, snowball)
text = vectorizer.transform([text])

# Obtenemos la predicción
pred = model_tree.predict(text)

# Obtenemos el intent en base a la predicción
intent = encoder.inverse_transform(pred)[0]
print("Intent: ", intent)

# Generamos un utterance de manera aleatoria
print("Respuesta 1: ", choose_utterance(intent, utterances))
print("Respuesta 2: ", choose_utterance(intent, utterances))

Intent:  sintoma_gripe
Respuesta 1:  para entender mejor lo que necesitas por favor compartenos tus sintomas procura que tu respuesta sea concisa
Respuesta 2:  para entender mejor lo que necesitas por favor compartenos tus sintomas procura que tu respuesta sea concisa


In [ ]:
# preprocesamos el texto
text = "muchas gracias por todo, nos vemos"
text = preprocess(text)
text = stem_text(text, snowball)
text = vectorizer.transform([text])

# Obtenemos la predicción
pred = model_tree.predict(text)

# Obtenemos el intent en base a la predicción
intent = encoder.inverse_transform(pred)[0]
print("Intent: ", intent)

# Generamos un utterance de manera aleatoria
print("Respuesta 1: ", choose_utterance(intent, utterances))
print("Respuesta 2: ", choose_utterance(intent, utterances))

Intent:  despedida
Respuesta 1:  todo listo para tu cita un gusto conversar contigo
Respuesta 2:  muchas gracias por confiar en nosotros hasta luego


In [ ]:
# preprocesamos el texto
text = "buena información, gracias"
text = preprocess(text)
text = stem_text(text, snowball)
text = vectorizer.transform([text])

# Obtenemos la predicción
pred = model_tree.predict(text)

print(model_tree.predict_proba(text))
# Obtenemos el intent en base a la predicción
intent = encoder.inverse_transform(pred)[0]
print("Intent: ", intent)

# Generamos un utterance de manera aleatoria
print("Respuesta 1: ", choose_utterance(intent, utterances))
print("Respuesta 2: ", choose_utterance(intent, utterances))

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]]
Intent:  despedida
Respuesta 1:  eso fue rapido tu cita esta agendada hasta luego
Respuesta 2:  eso fue rapido tu cita esta agendada hasta luego


## RASA NLU

In [ ]:
#@title nlu_data.md
%%writefile nlu_data.md
{
	"rasa_nlu_data": {
		"common_examples": [{
			"text": "hola",
			"intent": "saludo",
			"entities": []
		}, {
			"text": "hola que tal",
			"intent": "saludo",
			"entities": []
		}, {
			"text": "buenas tardes",
			"intent": "saludo",
			"entities": []
		}, {
			"text": "buenos dias",
			"intent": "saludo",
			"entities": []
		}, {
			"text": "buenas noches",
			"intent": "saludo",
			"entities": []
		}, {
			"text": "hola tengo una consulta",
			"intent": "saludo",
			"entities": []
		}, {
			"text": "quisiera consultar algo",
			"intent": "saludo",
			"entities": []
		}, {
			"text": "hola quiero saber si me pueden ayudar",
			"intent": "saludo",
			"entities": []
		}, {
			"text": "que tal tengo una consulta",
			"intent": "saludo",
			"entities": []
		}, {
			"text": "hasta luego",
			"intent": "despedida",
			"entities": []
		}, {
			"text": "hasta pronto",
			"intent": "despedida",
			"entities": []
		}, {
			"text": "fue un gusto conversar",
			"intent": "despedida",
			"entities": []
		}, {
			"text": "super bueno el apoyo",
			"intent": "despedida",
			"entities": []
		}, {
			"text": "por el momento no gracias",
			"intent": "despedida",
			"entities": []
		}, {
			"text": "no deseo gracias",
			"intent": "despedida",
			"entities": []
		}, {
			"text": "chau muchas gracias",
			"intent": "despedida",
			"entities": []
		}, {
			"text": "fue util la informacion",
			"intent": "despedida",
			"entities": []
		}, {
			"text": "nos vemos",
			"intent": "despedida",
			"entities": []
		}, {
			"text": "chau",
			"intent": "despedida",
			"entities": []
		}, {
			"text": "adios",
			"intent": "despedida",
			"entities": []
		}, {
			"text": "gracias por todo",
			"intent": "despedida",
			"entities": []
		}, {
			"text": "eso es todo muchas gracias",
			"intent": "despedida",
			"entities": []
		}, {
			"text": "gracias por la ayuda",
			"intent": "despedida",
			"entities": []
		}, {
			"text": "me sirvio de mucho el apoyo",
			"intent": "despedida",
			"entities": []
		}, {
			"text": "gracias por la informacion",
			"intent": "despedida",
			"entities": []
		}, {
			"text": "en otro momento les escribo",
			"intent": "despedida",
			"entities": []
		}, {
			"text": "no deseo nada mas",
			"intent": "despedida",
			"entities": []
		}, {
			"text": "es todo por mi parte",
			"intent": "despedida",
			"entities": []
		}, {
			"text": "tengo los ojos irritados",
			"intent": "sintoma_alergia",
			"entities": []
		}, {
			"text": "tengo los ojos rojos",
			"intent": "sintoma_alergia",
			"entities": []
		}, {
			"text": "tengo los ojos llorosos",
			"intent": "sintoma_alergia",
			"entities": []
		}, {
			"text": "picazon de ojos",
			"intent": "sintoma_alergia",
			"entities": []
		}, {
			"text": "dolor de ojos",
			"intent": "sintoma_alergia",
			"entities": []
		}, {
			"text": "me duelen ambos ojos",
			"intent": "sintoma_alergia",
			"entities": []
		}, {
			"text": "tengo muchos estornudos",
			"intent": "sintoma_alergia",
			"entities": []
		}, {
			"text": "nariz tapada",
			"intent": "sintoma_alergia",
			"entities": []
		}, {
			"text": "nariz con mocos",
			"intent": "sintoma_alergia",
			"entities": []
		}, {
			"text": "tengo la nariz llena de mocos",
			"intent": "sintoma_alergia",
			"entities": []
		}, {
			"text": "dolor de garganta",
			"intent": "sintoma_alergia",
			"entities": []
		}, {
			"text": "me duele mucho la garganta",
			"intent": "sintoma_alergia",
			"entities": []
		}, {
			"text": "picazon de garganta",
			"intent": "sintoma_alergia",
			"entities": []
		}, {
			"text": "carraspera",
			"intent": "sintoma_alergia",
			"entities": []
		}, {
			"text": "tengo tos constante",
			"intent": "sintoma_alergia",
			"entities": []
		}, {
			"text": "sensacion de cansancio",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "estoy muy cansado",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "mareos",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "tengo mareos",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "me siento mareado",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "aturdimiento",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "estoy aturdido",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "me he sentido aturdido",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "debilidad",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "me siento debil",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "me siento muy debil",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "fatiga",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "estoy fatigado",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "me siento fatigado",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "energia disminuida",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "me siento con poca energia",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "no tengo energia",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "piel palida",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "estoy palido",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "me veo palido",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "palidez",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "piel amarillenta",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "tengo la piel amarrilla",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "mi piel esta amarilla",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "el color de mi piel es amarillo",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "palpitaciones",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "siento palpitaciones extranas",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "el corazon me palpita de forma irregular",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "mi corazon palpita de forma rara",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "latidos del corazon irregulaes",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "siento latidos del corazon irregulaes",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "creo que tengo frecuencia cardiaca alta",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "tengo frecuencia cardiaca alta",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "frecuencia cardiaca alta",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "dificultad para respirar",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "me cuesta mucho respirar",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "me falta el aire constantemente",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "dolor en el pecho",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "siento dolor en el pecho",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "mis manos y pies estan frios",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "siento mis pies frios",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "mis manos estan frias",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "manos y pies frios",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "dolores de cabeza",
			"intent": "sintoma_anemia",
			"entities": []
		}, {
			"text": "me duele la cabeza",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "me duele mucho la cabeza",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "tengo dolor de cabeza",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "dolor de cabeza",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "dolor muscular",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "me duelen los musculos",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "me duele todo el cuerpo",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "tengo dolor muscular",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "escalofrios",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "siento escalofrios",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "me dan escalofrios",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "perdida de apetito",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "no tengo apetito",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "dolor de cuerpo",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "dolor generalizado de cuerpo",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "fiebre",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "tengo fiebre",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "tengo temperatura",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "tengo temperatura alta",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "congestion nasal",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "tengo mocos",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "tengo la nariz con mocos",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "nariz con mocos",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "tengo secrecion nasal",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "secrecion nasal",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "estornudos",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "estoy con estornudos",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "tos",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "tengo tos",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "dolor de garganta",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "me duele la garganta",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "me duele mucho la garganta",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "sudoracion",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "sudo mucho",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "ganglios linfaticos inflamados",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "mis ganglios linfaticos estan inflamados",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "me duele la garganta",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "me duelen mis ganglios",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "nauseas",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "tengo nauseas",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "me dan ganas de vomitar",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "ganas de vomitar",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "sensasion de vomitos",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "presion en el pecho",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "siento presion en el pecho",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "me presiona el pecho",
			"intent": "sintoma_gripe",
			"entities": []
		}, {
			"text": "dificultad para respirar",
			"intent": "sintoma_asma",
			"entities": []
		}, {
			"text": "falta de aire",
			"intent": "sintoma_asma",
			"entities": []
		}, {
			"text": "dolor de pecho",
			"intent": "sintoma_asma",
			"entities": []
		}, {
			"text": "opresion en el pecho",
			"intent": "sintoma_asma",
			"entities": []
		}, {
			"text": "tos",
			"intent": "sintoma_asma",
			"entities": []
		}, {
			"text": "pitido al respirar",
			"intent": "sintoma_asma",
			"entities": []
		}, {
			"text": "silbido al respirar",
			"intent": "sintoma_asma",
			"entities": []
		}, {
			"text": "perdida de orientacion",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "ultimamente he perdido el sentido de la orientacion",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "me olvido del lugar en el que estoy",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "suelo olvidarme donde estoy",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "a veces no se donde estoy",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "confusion de lugares",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "suelo confundir los lugares",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "agitacion",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "me siento agitado",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "me agito rapidamente",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "cambios de humor",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "tengo cambios de humor repentino",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "me cuesta planificar",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "me es dificil planificarme",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "dificultad para planificar",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "a veces me cuesta planificar mis cosas",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "dificultad para concentrarse",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "no puedo concentrarme",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "a veces me cuesta concentrarme",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "dificultad para resolver problemas",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "a veces tengo dificiltad para resolver problemas",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "dificultad para recordar eventos",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "me cuesta recordar eventos",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "me falla la memoria",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "no me acuerdo de algunas cosas",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "capacidad menor para cuidarse a si mismo",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "no puedo cuidarme por mi cuenta",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "problemas para encontrar palabras al hablar",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "a veces me trabo y no se que decir",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "me cuesta decir lo que pienso al hablar",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "depresion repentina",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "me siento deprimido",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "ultimamente me siento deprimido",
			"intent": "sintoma_alzeimer",
			"entities": []
		}, {
			"text": "ausencia de periodos menstruales",
			"intent": "sintoma_embarazo",
			"entities": []
		}, {
			"text": "no he tenido mi periodo menstrual",
			"intent": "sintoma_embarazo",
			"entities": []
		}, {
			"text": "hinchazon y sensibilidad en las mamas",
			"intent": "sintoma_embarazo",
			"entities": []
		}, {
			"text": "siento dolor en las mamas",
			"intent": "sintoma_embarazo",
			"entities": []
		}, {
			"text": "hinchazon en las mamas",
			"intent": "sintoma_embarazo",
			"entities": []
		}, {
			"text": "sensibilidad en las mamas",
			"intent": "sintoma_embarazo",
			"entities": []
		}, {
			"text": "nauseas y vomitos",
			"intent": "sintoma_embarazo",
			"entities": []
		}, {
			"text": "tengo nauseas",
			"intent": "sintoma_embarazo",
			"entities": []
		}, {
			"text": "tengo ganas de vomitar",
			"intent": "sintoma_embarazo",
			"entities": []
		}, {
			"text": "antojos",
			"intent": "sintoma_embarazo",
			"entities": []
		}, {
			"text": "tengo antojos repentinos",
			"intent": "sintoma_embarazo",
			"entities": []
		}, {
			"text": "aversiones a la comida",
			"intent": "sintoma_embarazo",
			"entities": []
		}, {
			"text": "no soporto el olor a comida",
			"intent": "sintoma_embarazo",
			"entities": []
		}, {
			"text": "no me gusta el olo de la comida",
			"intent": "sintoma_embarazo",
			"entities": []
		}, {
			"text": "fatiga",
			"intent": "sintoma_embarazo",
			"entities": []
		}, {
			"text": "siento fatiga",
			"intent": "sintoma_embarazo",
			"entities": []
		}, {
			"text": "me siento cansado",
			"intent": "sintoma_embarazo",
			"entities": []
		}, {
			"text": "estoy consada",
			"intent": "sintoma_embarazo",
			"entities": []
		}, {
			"text": "dolores de cabeza",
			"intent": "sintoma_embarazo",
			"entities": []
		}, {
			"text": "me duele la cabeza",
			"intent": "sintoma_embarazo",
			"entities": []
		}, {
			"text": "cambios en el estado de animo",
			"intent": "sintoma_embarazo",
			"entities": []
		}, {
			"text": "miccion frecuente",
			"intent": "sintoma_embarazo",
			"entities": []
		}, {
			"text": "voy al bano frecuentemente",
			"intent": "sintoma_embarazo",
			"entities": []
		}, {
			"text": "ganas de ir al bano constantemente",
			"intent": "sintoma_embarazo",
			"entities": []
		}, {
			"text": "frecuencia cardiaca rapida",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "me late el corazon muy rapido",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "el corazon me late muy de prisa",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "sudoracion excesiva",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "sudo mucho",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "intolerancia al calor",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "no soporto el calor",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "no soporto altas temperaturas",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "me fastidia el calor",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "temblores",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "siento temblores en el cuerpo",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "nerviosismo o agitacion",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "me agito mucho",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "me siento agitado constantemente",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "fatiga",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "me siento fatigado",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "siento fatiga",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "perdida de peso",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "he bajado de peso muy rapido",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "he bajado de peso",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "he perdido peso",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "caida del cabello",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "se me cae el cabello",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "se me cae el pelo",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "pierdo cabello",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "me estoy quedando sin pelo",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "mucho apetito",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "tengo mucha hambre",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "tengo mas hambre de lo normal",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "problemas de concentracion",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "me cuesta concentrarme",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "deposiciones frecuentes",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "voy al bano constantemente",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "voy al bano a cada rato",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "voy al bano frecuentemente",
			"intent": "sintoma_hipertiroidismo",
			"entities": []
		}, {
			"text": "fatiga",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "me siento fatigado",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "siento fatiga",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "depresion",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "me siento deprimido",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "estoy con depresion",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "leve aumento de peso",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "he subido de peso",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "intolerancia al frio",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "no soporto el frio",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "ultimamente no soporto el frio",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "no puedo estar en lugares frios",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "somnolencia",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "tengo muchas ganas de dormir",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "me cuesta manenerme despierto en el dia",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "tengo mucho sueno durante el dia",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "estrenimiento",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "estoy estrenido",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "cabello seco y aspero",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "mi cabello esta seco",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "mi cabello esta aspero",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "mi pelo esta seco",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "mi pelo esta aspero",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "piel seca y calambres musculares",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "tengo la piel seca",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "tengo calambres musculares",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "calambres en los musculos",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "niveles de colesterol elevados en la sangre",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "tengo el colesterol elevado",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "tengo el nivel de colesterol alto",
			"intent": "sintoma_hipotiroidismo",
			"entities": []
		}, {
			"text": "mucha sed",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "tengo mucha sed",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "tomo mucha agua durante el dia",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "me siendo deshidratado",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "deshidratacion",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "estoy deshidratado",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "hambre",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "tengo mucha hambre",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "no siendo saciedad",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "ganas de miccionar",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "tengo muchas ganas de ir al bano",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "voy al bano muchas veces durante el dia",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "me dan ganas de ir al bano muchas veces en el dia",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "fatiga",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "me siento fatigado",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "siento fatiga",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "vision borrosa",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "veo borroso",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "no veo bien",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "perdida de peso inesperada",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "he bajado de peso de forma repentina",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "mucha hambre",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "tengo mucha hambre",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "aumento de apetito",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "llagas de curacion lenta",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "mis heridas se curan lentamente",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "infecciones frecuentes",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "tengo infecciones frecuentes",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "encias rojas e inflamadas",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "encias inflamadas",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "encias rojas",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "hormigueo o entumecimiento en las manos o los pies",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "sensacion de entumecimiento en los pies",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "sensacion de entumecimiento en las manos",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "sensacion de hormigueo en las manos",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "sensacion de hormigueo en los pies",
			"intent": "sintoma_diabetes",
			"entities": []
		}, {
			"text": "fiebre",
			"intent": "sintoma_sida",
			"entities": []
		}, {
			"text": "inflamacion de los ganglios linfaticos",
			"intent": "sintoma_sida",
			"entities": []
		}, {
			"text": "ganglios inflamados",
			"intent": "sintoma_sida",
			"entities": []
		}, {
			"text": "dolores musculares y articulares",
			"intent": "sintoma_sida",
			"entities": []
		}, {
			"text": "dolor de cabeza",
			"intent": "sintoma_sida",
			"entities": []
		}, {
			"text": "dolor de garganta",
			"intent": "sintoma_sida",
			"entities": []
		}, {
			"text": "escalofrios",
			"intent": "sintoma_sida",
			"entities": []
		}, {
			"text": "sudoracion nocturna",
			"intent": "sintoma_sida",
			"entities": []
		}, {
			"text": "ulceras en la boca",
			"intent": "sintoma_sida",
			"entities": []
		}, {
			"text": "mareos",
			"intent": "sintoma_hipertension",
			"entities": []
		}, {
			"text": "dificultad para respirar",
			"intent": "sintoma_hipertension",
			"entities": []
		}, {
			"text": "dolor de cabeza fuerte",
			"intent": "sintoma_hipertension",
			"entities": []
		}, {
			"text": "nauseas",
			"intent": "sintoma_hipertension",
			"entities": []
		}, {
			"text": "vomitos",
			"intent": "sintoma_hipertension",
			"entities": []
		}, {
			"text": "vision borrosa",
			"intent": "sintoma_hipertension",
			"entities": []
		}, {
			"text": "hemorragias nasales",
			"intent": "sintoma_hipertension",
			"entities": []
		}, {
			"text": "sangre en la orina",
			"intent": "sintoma_hipertension",
			"entities": []
		}, {
			"text": "fatiga",
			"intent": "sintoma_hipertension",
			"entities": []
		}, {
			"text": "sangrado nasal",
			"intent": "sintoma_hipertension",
			"entities": []
		}, {
			"text": "dolor muscular",
			"intent": "sintoma_artritis",
			"entities": []
		}, {
			"text": "dolor articular",
			"intent": "sintoma_artritis",
			"entities": []
		}, {
			"text": "inflamacion articular",
			"intent": "sintoma_artritis",
			"entities": []
		}, {
			"text": "rigidez articular",
			"intent": "sintoma_artritis",
			"entities": []
		}, {
			"text": "fatiga",
			"intent": "sintoma_artritis",
			"entities": []
		}, {
			"text": "falta de apetito",
			"intent": "sintoma_artritis",
			"entities": []
		}, {
			"text": "perdida de energia",
			"intent": "sintoma_artritis",
			"entities": []
		}, {
			"text": "articulaciones hinchadas",
			"intent": "sintoma_artritis",
			"entities": []
		}, {
			"text": "fatiga",
			"intent": "sintoma_hepatitisB",
			"entities": []
		}, {
			"text": "nauseas",
			"intent": "sintoma_hepatitisB",
			"entities": []
		}, {
			"text": "dolor en el abdomen",
			"intent": "sintoma_hepatitisB",
			"entities": []
		}, {
			"text": "dolor abdominal",
			"intent": "sintoma_hepatitisB",
			"entities": []
		}, {
			"text": "orina oscura",
			"intent": "sintoma_hepatitisB",
			"entities": []
		}, {
			"text": "dolor articular",
			"intent": "sintoma_hepatitisB",
			"entities": []
		}, {
			"text": "perdida de apetito",
			"intent": "sintoma_hepatitisB",
			"entities": []
		}, {
			"text": "debilidad y fatiga",
			"intent": "sintoma_hepatitisB",
			"entities": []
		}, {
			"text": "piel amarilla",
			"intent": "sintoma_hepatitisB",
			"entities": []
		}, {
			"text": "parte blanca del ojo amarilla",
			"intent": "sintoma_hepatitisB",
			"entities": []
		}, {
			"text": "dolor de cabeza",
			"intent": "sintoma_dengue",
			"entities": []
		}, {
			"text": "fiebre",
			"intent": "sintoma_dengue",
			"entities": []
		}, {
			"text": "agotamiento",
			"intent": "sintoma_dengue",
			"entities": []
		}, {
			"text": "dolor severo en las articulaciones de los musculos",
			"intent": "sintoma_dengue",
			"entities": []
		}, {
			"text": "inflamacion de los ganglios linfaticos",
			"intent": "sintoma_dengue",
			"entities": []
		}, {
			"text": "erupcion cutanea",
			"intent": "sintoma_dengue",
			"entities": []
		}, {
			"text": "dolor en los ojos",
			"intent": "sintoma_dengue",
			"entities": []
		}, {
			"text": "fiebre",
			"intent": "sintoma_malaria",
			"entities": []
		}, {
			"text": "escalofrios",
			"intent": "sintoma_malaria",
			"entities": []
		}, {
			"text": "nauseas",
			"intent": "sintoma_malaria",
			"entities": []
		}, {
			"text": "vomitos",
			"intent": "sintoma_malaria",
			"entities": []
		}, {
			"text": "heces con sangre",
			"intent": "sintoma_malaria",
			"entities": []
		}, {
			"text": "sudoracion",
			"intent": "sintoma_malaria",
			"entities": []
		}, {
			"text": "debilidad general",
			"intent": "sintoma_varicela",
			"entities": []
		}, {
			"text": "fiebre muy alta",
			"intent": "sintoma_varicela",
			"entities": []
		}, {
			"text": "manchas rojas en la piel",
			"intent": "sintoma_varicela",
			"entities": []
		}, {
			"text": "manchas rojas y fiebre",
			"intent": "sintoma_varicela",
			"entities": []
		}, {
			"text": "cansancio y sensacion de malestar general",
			"intent": "sintoma_varicela",
			"entities": []
		}, {
			"text": "ojos amarillos",
			"intent": "sintoma_ictericia",
			"entities": []
		}, {
			"text": "decoloracion amarilla de la piel",
			"intent": "sintoma_ictericia",
			"entities": []
		}, {
			"text": "coloracion amarilla de la piel",
			"intent": "sintoma_ictericia",
			"entities": []
		}, {
			"text": "piel clara",
			"intent": "sintoma_ictericia",
			"entities": []
		}, {
			"text": "coloracion amarilla dentro de la boca",
			"intent": "sintoma_ictericia",
			"entities": []
		}, {
			"text": "somnolencia excesiva",
			"intent": "sintoma_ictericia",
			"entities": []
		}, {
			"text": "orina oscura o de color marron",
			"intent": "sintoma_ictericia",
			"entities": []
		}, {
			"text": "heces palidas",
			"intent": "sintoma_ictericia",
			"entities": []
		}, {
			"text": "heces de color arcilla",
			"intent": "sintoma_ictericia",
			"entities": []
		}, {
			"text": "deposiciones flojas y acuosas",
			"intent": "sintoma_diarrea",
			"entities": []
		}, {
			"text": "calambres abdominales",
			"intent": "sintoma_diarrea",
			"entities": []
		}, {
			"text": "dolor de estomago",
			"intent": "sintoma_diarrea",
			"entities": []
		}, {
			"text": "calambres y dolor abdominal",
			"intent": "sintoma_diarrea",
			"entities": []
		}, {
			"text": "fiebre",
			"intent": "sintoma_diarrea",
			"entities": []
		}, {
			"text": "sangre en las heces",
			"intent": "sintoma_diarrea",
			"entities": []
		}, {
			"text": "hinchazon",
			"intent": "sintoma_diarrea",
			"entities": []
		}, {
			"text": "nauseas",
			"intent": "sintoma_diarrea",
			"entities": []
		}, {
			"text": "disminucion de la claridad de la vision",
			"intent": "sintoma_catarata",
			"entities": []
		}, {
			"text": "incapacidad para ver con luz tenue",
			"intent": "sintoma_catarata",
			"entities": []
		}, {
			"text": "perdida de sensibilidad al contraste",
			"intent": "sintoma_catarata",
			"entities": []
		}, {
			"text": "perdida de la vision",
			"intent": "sintoma_catarata",
			"entities": []
		}, {
			"text": "ver halos alrededor de las luces",
			"intent": "sintoma_catarata",
			"entities": []
		}, {
			"text": "deslumbramiento perturbador en la luz",
			"intent": "sintoma_catarata",
			"entities": []
		}, {
			"text": "vision borrosa",
			"intent": "sintoma_catarata",
			"entities": []
		}, {
			"text": "fiebre",
			"intent": "sintoma_neumonia",
			"entities": []
		}, {
			"text": "resfriado",
			"intent": "sintoma_neumonia",
			"entities": []
		}, {
			"text": "escalofrios",
			"intent": "sintoma_neumonia",
			"entities": []
		}, {
			"text": "tos",
			"intent": "sintoma_neumonia",
			"entities": []
		}, {
			"text": "dificultad para respirar",
			"intent": "sintoma_neumonia",
			"entities": []
		}, {
			"text": "fatiga",
			"intent": "sintoma_neumonia",
			"entities": []
		}, {
			"text": "incapacidad para orinar",
			"intent": "sintoma_retencionurinaria",
			"entities": []
		}, {
			"text": "flujo de orina es debil e interrmpido",
			"intent": "sintoma_retencionurinaria",
			"entities": []
		}, {
			"text": "malestar en la parte inferior del abdomen",
			"intent": "sintoma_retencionurinaria",
			"entities": []
		}, {
			"text": "dolor en la parte inferior del abdomen",
			"intent": "sintoma_retencionurinaria",
			"entities": []
		}, {
			"text": "hinchazon de la parte inferior del abdomen",
			"intent": "sintoma_retencionurinaria",
			"entities": []
		}, {
			"text": "cansancio",
			"intent": "sintoma_insomnia",
			"entities": []
		}, {
			"text": "fatiga diurna",
			"intent": "sintoma_insomnia",
			"entities": []
		}, {
			"text": "cambios de humor",
			"intent": "sintoma_insomnia",
			"entities": []
		}, {
			"text": "poca concentracion",
			"intent": "sintoma_insomnia",
			"entities": []
		}, {
			"text": "ansiedad",
			"intent": "sintoma_insomnia",
			"entities": []
		}, {
			"text": "dolor de cabeza",
			"intent": "sintoma_insomnia",
			"entities": []
		}, {
			"text": "falta de energia",
			"intent": "sintoma_insomnia",
			"entities": []
		}, {
			"text": "fiebre",
			"intent": "sintoma_covid",
			"entities": []
		}, {
			"text": "tos seca",
			"intent": "sintoma_covid",
			"entities": []
		}, {
			"text": "cansancio",
			"intent": "sintoma_covid",
			"entities": []
		}, {
			"text": "molestias y dolores",
			"intent": "sintoma_covid",
			"entities": []
		}, {
			"text": "dolor de garganta",
			"intent": "sintoma_covid",
			"entities": []
		}, {
			"text": "diarrea",
			"intent": "sintoma_covid",
			"entities": []
		}, {
			"text": "conjuntivitis",
			"intent": "sintoma_covid",
			"entities": []
		}, {
			"text": "perdida del sentido del olfato o del gusto",
			"intent": "sintoma_covid",
			"entities": []
		}, {
			"text": "dificultad para respirar",
			"intent": "sintoma_covid",
			"entities": []
		}, {
			"text": "falta de aire",
			"intent": "sintoma_covid",
			"entities": []
		}, {
			"text": "dolor o presion en el pecho",
			"intent": "sintoma_covid",
			"entities": []
		}, {
			"text": "incapacidad para hablar o moverse",
			"intent": "sintoma_covid",
			"entities": []
		}]
	}
}

Overwriting nlu_data.md


In [ ]:
#@title nlu-config.yml
%%writefile nlu-config.yml
language: "es"

pipeline:
- name: "SpacyNLP"
- name: "SpacyTokenizer"
- name: "SpacyFeaturizer"
- name: "RegexFeaturizer"
- name: "CRFEntityExtractor"
- name: "EntitySynonymMapper"
- name: "SklearnIntentClassifier"

In [ ]:
#@title nlu-config-large.yml
%%writefile nlu-config-large.yml
language: "es"

pipeline:
  - name: SpacyNLP
  - name: SpacyTokenizer
  - name: SpacyFeaturizer
  - name: RegexFeaturizer
  - name: CountVectorsFeaturizer
  - name: CountVectorsFeaturizer
    analyzer: "char_wb"
    min_ngram: 1
    max_ngram: 4
  - name: EntitySynonymMapper

Overwriting nlu-config-large.yml


In [ ]:
def create_nlu_data():
    text = """{"rasa_nlu_data": { "common_examples":"""
    content = ""
    for intent, values in intents.items():
        for v in values:
            content += f'{"{"} "text": "{v}", "intent": "{intent}" , "entities": [] {"},"}'

    items = "[" + content[:-1] + "]"
    all = text + items + "}}"
    return all
    #print(all)
    #with open("nlu_data.md", 'w') as f:
    #    json.dump(all, f)

In [ ]:
# import rasa_nlu
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

training_data = load_data("nlu_data.md")

trainer = Trainer(config.load("nlu-config.yml"))

interpreter = trainer.train(training_data)

model_directory = trainer.persist("./models/nlu", fixed_model_name="my_model")

INFO:rasa_nlu.training_data.loading:Training data format of nlu_data.md is rasa_nlu
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 407 (25 distinct intents)
	- Found intents: 'sintoma_covid', 'saludo', 'sintoma_asma', 'sintoma_sida', 'despedida', 'sintoma_hipertiroidismo', 'sintoma_varicela', 'sintoma_hipotiroidismo', 'sintoma_alzeimer', 'sintoma_gripe', 'sintoma_neumonia', 'sintoma_embarazo', 'sintoma_diarrea', 'sintoma_retencionurinaria', 'sintoma_anemia', 'sintoma_artritis', 'sintoma_alergia', 'sintoma_insomnia', 'sintoma_diabetes', 'sintoma_catarata', 'sintoma_ictericia', 'sintoma_dengue', 'sintoma_malaria', 'sintoma_hepatitisB', 'sintoma_hipertension'
	- entity examples: 0 (0 distinct entities)
	- found entities: 

INFO:rasa_nlu.utils.spacy_utils:Trying to load spacy model with name 'es'
INFO:rasa_nlu.components:Added 'SpacyNLP' to component cache. Key 'SpacyNLP-es'.
INFO:rasa_nlu.model:Starting to train component SpacyNLP
INFO:rasa_nlu.model:F

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.8s finished
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Successfully saved model into '/content/models/nlu/default/my_model'


In [ ]:
interpreter.parse("me falta el aire al respirar y escucho un silbido en el pecho cuando respiro")

{'entities': [],
 'intent': {'confidence': 0.14865288856252665,
  'name': 'sintoma_hipertiroidismo'},
 'intent_ranking': [{'confidence': 0.14865288856252665,
   'name': 'sintoma_hipertiroidismo'},
  {'confidence': 0.13072726732914486, 'name': 'sintoma_anemia'},
  {'confidence': 0.0994205557174567, 'name': 'sintoma_gripe'},
  {'confidence': 0.0671499710308085, 'name': 'sintoma_covid'},
  {'confidence': 0.06518722774308254, 'name': 'sintoma_embarazo'},
  {'confidence': 0.0642928414856173, 'name': 'sintoma_hipotiroidismo'},
  {'confidence': 0.049284181909271974, 'name': 'sintoma_diabetes'},
  {'confidence': 0.04905329782829971, 'name': 'sintoma_asma'},
  {'confidence': 0.03681846520099988, 'name': 'sintoma_alzeimer'},
  {'confidence': 0.03254281261343895, 'name': 'despedida'}],
 'text': 'me falta el aire al respirar y escucho un silbido en el pecho cuando respiro'}

In [ ]:
interpreter.parse("muchas gracias, hasta luego")

{'entities': [],
 'intent': {'confidence': 0.20107369898308008, 'name': 'despedida'},
 'intent_ranking': [{'confidence': 0.20107369898308008, 'name': 'despedida'},
  {'confidence': 0.1730440406658987, 'name': 'sintoma_alzeimer'},
  {'confidence': 0.10609269515409625, 'name': 'sintoma_diabetes'},
  {'confidence': 0.07488809192886721, 'name': 'sintoma_hipertiroidismo'},
  {'confidence': 0.06973007932443548, 'name': 'sintoma_embarazo'},
  {'confidence': 0.053087511171699166, 'name': 'sintoma_hipotiroidismo'},
  {'confidence': 0.044858824672864436, 'name': 'sintoma_gripe'},
  {'confidence': 0.04034433127661588, 'name': 'sintoma_anemia'},
  {'confidence': 0.021070901281142237, 'name': 'sintoma_sida'},
  {'confidence': 0.018693714125320844, 'name': 'sintoma_dengue'}],
 'text': 'muchas gracias, hasta luego'}

In [ ]:
model = None
with open('/content/models/nlu/default/my_model/component_6_SklearnIntentClassifier.pkl', 'rb') as f:
    model = pickle.load(f) 

In [ ]:
with open('interpreter.pkl', 'wb') as f:
    pickle.dump(interpreter, f) 